In [7]:
import os
import json
import itertools
from domain_groups import group_domains
from collections import defaultdict, Counter
DIR = "../optimization/"

DOMAINS = [
    "barman", "blocksworld", "childsnack", "data-network", "depots", "driverlog",
    "elevators", "floortile", "grid", "gripper", "hiking", "logistics",
    "miconic", "nomystery", "openstacks", "parking", "rovers", "satellite",
    "scanalyzer", "snake", "storage", "tpp", "transport", "visitall",
    "woodworking", "zenotravel",
]

TRAINING_PLANNERS_OPT = [
    "fd1906-blind",
    "fd1906-fdss1-mas1-60s",
    "fd1906-fdss1-mas2-60s",
    "fd1906-bjolp",
    "fd1906-lmcut",
    "ipc2014-opt-symba1",
]
EVALUATION_PLANNERS_OPT = [
    "ipc2018-opt-complementary2-3584mb",
    "ipc2018-opt-decstar",
    "ipc2018-opt-delfi-blind",
    "ipc2018-opt-delfi-celmcut",
    "ipc2018-opt-delfi-ipdb",
    "ipc2018-opt-delfi-mas-miasm",
    "ipc2018-opt-delfi-mas-sccdfp",
    "ipc2018-opt-scorpion",
]

TRAINING_PLANNERS_SAT = [
    "fd1906-gbfs-ff",
    "fd1906-lama-first",
    "ipc2014-agl-mpc",
    "ipc2014-agl-mercury",
    "ipc2014-agl-jasper",
    "ipc2014-agl-probe",
]
EVALUATION_PLANNERS_SAT = [
    "ipc2018-agl-decstar",
    "ipc2018-agl-fd-remix",
    "ipc2018-agl-lapkt-dual-bfws",
    "ipc2018-agl-lapkt-bfws-pref",
    "ipc2018-agl-lapkt-poly-bfws",
    "ipc2018-agl-olcff",
    "ipc2018-agl-saarplan",
    "ipc2018-agl-cerberus",
]

TRAINING_PLANNERS = {
        "opt-ipc" : TRAINING_PLANNERS_OPT,
        "opt-1204": TRAINING_PLANNERS_OPT,
        "sat-ipc": TRAINING_PLANNERS_SAT,
        "sat-1204": TRAINING_PLANNERS_SAT,
}

EVALUATION_PLANNERS = {
        "opt-ipc" : EVALUATION_PLANNERS_OPT,
        "opt-1204": EVALUATION_PLANNERS_OPT,
        "sat-ipc": EVALUATION_PLANNERS_SAT,
        "sat-1204": EVALUATION_PLANNERS_SAT,
}

FILENAMES = {
        "opt-ipc": ["2020-11-23-A-optimization-planners-ipc-properties.json", "2020-12-05-A-evaluation-opt-ipc-properties.json"],
        "opt-1204": ["2020-12-05-C-evaluation-opt-new2014-properties.json"],
        "sat-ipc": ["2020-11-23-D-optimization-planners-sat-ipc-properties.json","2020-12-05-B-evaluation-sat-ipc-properties.json"],
        "sat-1204": ["2020-12-05-D-evaluation-sat-new2014-properties.json"]
    }

def read_runs(filename):
    with open(filename) as f:
        data = json.load(f)
        for run, values in data.items(): 
            if "coverage" not in values: 
                #print ("Warning, coverage attribute missing in ", filename, run)
                pass
        return data.values()

# Step 1: gather all runs and label them by dataset and domain
all_runs = defaultdict(list)
for name, filenames in FILENAMES.items():
    
    for filename in filenames: 
        new_runs = read_runs(os.path.join(DIR, "results", filename))
        new_runs = [group_domains(run) for run in new_runs]
        new_runs = [run for run in new_runs if run["domain"] in DOMAINS]
        
        for run in new_runs: 
            run["dataset"] = name
            all_runs[(name, run["domain"])].append(run)
        
        

In [8]:
def compute_coverage (properties, atr_name, runs, time_limit=100000):
    coverage = Counter()
    for run in runs: 
        if "coverage" in run and run["coverage"] == 1 and run["planner_wall_clock_time"] <= time_limit: 
            coverage[run["algorithm"]] += 1
    properties[atr_name] = coverage
    

def compute_comparisons (properties, comp_atr_name, cov_atr_name, planners):
    coverage = properties[cov_atr_name]
    properties[comp_atr_name] = sum ([1 for (p1, p2) in itertools.combinations(planners, 2) if coverage[p1] != coverage[p2]])
    
    
def compute_runtimes (properties, atr_name, runs, planners):
    minimum_runtime = {}
    for run in runs: 
        if "coverage" in run and run["coverage"] == 1 and run["algorithm"] in planners: 
            problem_id = int(run["problem"].split("-p")[1].replace(".pddl", ""))
            if problem_id in minimum_runtime: 
                minimum_runtime[problem_id]= min(minimum_runtime[problem_id], run["planner_wall_clock_time"])
            else: 
                minimum_runtime[problem_id] = run["planner_wall_clock_time"]
                
    properties[atr_name] = [minimum_runtime[p] if p in minimum_runtime else "unsolved" for p in range(1, 31)]

    

# Step 2: Compute all aggregated properties for a dataset
properties_dataset = defaultdict(dict)
for dataset, domain in all_runs: 
    if "-ipc" in dataset: 
        continue
    ipcdataset = dataset.split("-")[0] + "-ipc"
    
    properties_dataset[(dataset, domain)] ["track"] = dataset
    properties_dataset[(dataset, domain)] ["domain"] = domain
    compute_coverage(properties_dataset[(dataset, domain)], "coverage", all_runs[(dataset, domain)])
    compute_coverage(properties_dataset[(dataset, domain)], "coverage30s", all_runs[(dataset, domain)], 30)
    compute_coverage(properties_dataset[(dataset, domain)], "coverage300s", all_runs[(dataset, domain)], 300)
        
    compute_runtimes(properties_dataset[(dataset, domain)], "runtimes-training", all_runs[(dataset, domain)], TRAINING_PLANNERS[dataset])
    compute_runtimes(properties_dataset[(dataset, domain)], "runtimes-eval", all_runs[(dataset, domain)], EVALUATION_PLANNERS[dataset])

    compute_coverage(properties_dataset[(dataset, domain)], "coverage-ipc", all_runs[(ipcdataset, domain)])
    compute_coverage(properties_dataset[(dataset, domain)], "coverage30s-ipc", all_runs[(ipcdataset, domain)], 30)
    compute_coverage(properties_dataset[(dataset, domain)], "coverage300s-ipc", all_runs[(ipcdataset, domain)], 300)

    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-training", "coverage", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-eval", "coverage", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-training", "coverage30s", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-eval", "coverage30s", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-training", "coverage300s", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-eval", "coverage300s", EVALUATION_PLANNERS[dataset])

    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-training-ipc", "coverage-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons-eval-ipc", "coverage-ipc", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-training-ipc", "coverage30s-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons30s-eval-ipc", "coverage30s-ipc", EVALUATION_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-training-ipc", "coverage300s-ipc", TRAINING_PLANNERS[dataset])
    compute_comparisons(properties_dataset[(dataset, domain)], "comparisons300s-eval-ipc", "coverage300s-ipc", EVALUATION_PLANNERS[dataset])
    
    for x in ["comparisons-training", "comparisons-eval", "comparisons300s-training", "comparisons300s-eval" ]:
        properties_dataset[(dataset, domain)][f"{x}-ipcdiff"] = properties_dataset[(dataset, domain)][x] - properties_dataset[(dataset, domain)][f"{x}-ipc"] 
    


In [9]:
LOGDIR = "../logfiles/2020-12-04/"
import os
import re

def parse_CPLEX_log(content):
    
    regex_int = [re.compile(x) for x in [".*Different evaluated sequences: (?P<evaluated_sequences>(\d+))",
                                     ".*Candidate sequences: (?P<candidate_sequences>(\d+))", 
                                    ]]
    regex_float = [re.compile(x) for x in [".*Solution value  = (?P<solution_value>(.*))",]]

    penalties = [float(l.split(":")[-1]) for l in content if l.startswith("Penalty: ")]
        
    data = {
            "sart_runtimes" : False, 
            "num_sequences" : 0, 
            "min_penalty" : min(penalties) if penalties else "-", 
            "max_penalty" : max(penalties) if penalties else "-",  
            "runtimes-estimated" : []
           }
    for l in content: 
        if "Using sart runtimes on CPLEX optimization" in l: 
            data["sart_runtimes"] = True
            continue
            
        if "Selected: sequence" in l:
            from_instance, to_instance = list(map(int, l.split("from")[1].split("to")))
            data["num_sequences"] += 1
            
        if l.startswith("Estimated runtimes:"):             
            runtimes_sequence = list(map(float, l.split(":")[1].split(",")))
            if len(runtimes_sequence) == 30:
                data["runtimes-estimated"] += runtimes_sequence[from_instance:to_instance+1]
            else:
                data["runtimes-estimated"] += runtimes_sequence[:to_instance-from_instance+1]
            

            
        for reg in regex_int: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = int(value)
        
        for reg in regex_float: 
            if reg.match(l): 
                for atr, value in reg.match(l).groupdict().items():
                    data [atr] = float(value)
                    
    assert len(data["runtimes-estimated"]) == 30, (data["runtimes-estimated"], len(data["runtimes-estimated"]))
    return data
    
for logfile in sorted(os.listdir(LOGDIR)):
    _, track, domain = logfile.split("_")
    
    dataset = track.replace("14", "-1204")
    
    f = open(f"{LOGDIR}/{logfile}")
    content = f.readlines()
    
    properties_dataset[(dataset, domain)] ["track"] = dataset
    properties_dataset[(dataset, domain)] ["domain"] = domain

    data = parse_CPLEX_log(content)
    properties_dataset[(dataset, domain)].update(data)
    



                 

In [10]:
import statistics
def compute_smoothness(properties, atr_name, runtimes):
    runtimes = sorted([r for r in runtimes if r != "unsolved"])
    
    factors = [r/runtimes[i] for i, r in enumerate (runtimes[1:])]
    runtimes_g5 = [r for r in runtimes if r > 5]
    factors_g5 = [r/runtimes_g5[i] for i, r in enumerate (runtimes_g5[1:])]
    
    properties [f"max_runtime_{atr_name}"] = max(runtimes) if runtimes else "-"
    properties [f"min_runtime_{atr_name}"] = min(runtimes) if runtimes else "-"
    
    properties [f"min_factor_runtime_{atr_name}"] = min(factors) if factors else "-"
    properties [f"max_factor_runtime_{atr_name}"] = max(factors) if factors else "-"
    properties [f"gmean_factor_runtime_{atr_name}"] = statistics.geometric_mean(factors) if factors else "-"
    
    properties [f"min_factorg5_runtime_{atr_name}"] = min(factors_g5) if factors_g5 else "-"
    properties [f"max_factorg5_runtime_{atr_name}"] = max(factors_g5) if factors_g5 else "-" 
    properties [f"gmean_factorg5_runtime_{atr_name}"] = statistics.geometric_mean(factors_g5) if factors_g5 else "-"
    
    return properties

for dataset, domain in properties_dataset: 
    properties_dataset[(dataset, domain)] = compute_smoothness(properties_dataset[(dataset, domain)], "estimated", properties_dataset[(dataset, domain)]["runtimes-estimated"])
    properties_dataset[(dataset, domain)] = compute_smoothness(properties_dataset[(dataset, domain)], "training", properties_dataset[(dataset, domain)]["runtimes-training"])
    properties_dataset[(dataset, domain)] = compute_smoothness(properties_dataset[(dataset, domain)], "eval", properties_dataset[(dataset, domain)]["runtimes-eval"])
        

In [11]:
import statistics

def estimated_error(properties):     
    properties["coverage-estimated"] = len([r   for r in properties["runtimes-estimated"] if r < 1800])
    properties["coverage-training"] = len([r  for r in properties["runtimes-training"] if r != "unsolved"])
    properties["estimated-error-coverage"] = abs(properties["coverage-estimated"] - properties["coverage-training"])

    properties["estimated-instance-error-coverage"] = 0
    properties["estimated-instance-error-runtime"] = 0
    paired_runtimes = zip([r if r < 1800 else "unsolved" for r in properties["runtimes-estimated"]], properties["runtimes-training"])
    for (est, tr) in paired_runtimes: 
        if est == tr: 
            continue
        if est == "unsolved" or tr == "unsolved":     
            properties["estimated-instance-error-coverage"] += 1
        else: 
            properties["estimated-instance-error-runtime"] += abs(est - tr)

    paired_sorted_runtimes = zip([r for r in sorted(properties["runtimes-estimated"]) if r < 1800], sorted([r for r in properties["runtimes-training"] if r != "unsolved"]))
    properties["estimated-error-runtime"] = statistics.geometric_mean([max(est,tr)/min(est, tr) for (est, tr) in paired_sorted_runtimes])

for dataset, domain in properties_dataset: 
    estimated_error(properties_dataset[(dataset, domain)])


In [13]:
def penalty_by_factor(factor):
    if factor <= 1:  # Runtime is not increasing: maximum penalty of 1
        return 1
    elif factor <= 1.5:
        return 3 - 2*factor
    elif factor <= 2: # Runtime is increasing, but not very quickly
        return 0
    elif factor > 2: # Runtime is increasing too quickly
        return 1 - (2 / factor)

def evaluate_runtimes_single_sequence(runtimes, num_expected_runtimes = 5):
    penalty = 0
    sorted_runtimes = sorted(runtimes)

    # The default scaling only works if all instances are solvable. For each unsolvable
    # instance apply a double penalty.
    if len(runtimes) < num_expected_runtimes:
        penalty += 2 * (num_expected_runtimes - len(runtimes))

    for i in range(1, len(runtimes)):
        factor = sorted_runtimes[i] / sorted_runtimes[i - 1]
        penalty += penalty_by_factor(factor)

    return penalty


def compute_runtime_penalty(properties, atr_name, runtimes): 
    s_runtimes = sorted([t for t in runtimes if t != "unsolved" and t >= 5]) [:5]
    penalty = evaluate_runtimes_single_sequence(s_runtimes)

    properties [f"{atr_name}"] = penalty

for dataset, domain in properties_dataset: 
    compute_runtime_penalty(properties_dataset[(dataset, domain)], "penalty-estimated", properties_dataset[(dataset, domain)]["runtimes-estimated"])
    compute_runtime_penalty(properties_dataset[(dataset, domain)], "penalty-training", properties_dataset[(dataset, domain)]["runtimes-training"])
    compute_runtime_penalty(properties_dataset[(dataset, domain)], "penalty-eval", properties_dataset[(dataset, domain)]["runtimes-eval"])


In [16]:
import pandas as pd

from IPython.display import display

def make_short(x): 
    return x.replace("comparisons", "cmp").replace("training", "tr")
ATRIBUTES_TABLE = ["track", "domain", "solution_value", "sart_runtimes", "comparisons-training", "comparisons-training-ipcdiff", "penalty-estimated", "penalty-training",  "penalty-eval",  "comparisons-eval", "comparisons-eval-ipcdiff", "estimated-error-coverage", "estimated-error-runtime", "coverage-estimated", "coverage-training"]

table = []
for data in sorted(properties_dataset.values(), key = lambda x : x["comparisons-eval-ipcdiff"] + x["comparisons-training-ipcdiff"]): 
    table.append([data[atr] if atr in data else "-" for atr in ATRIBUTES_TABLE ])

pd.options.display.float_format = "{:,.2g}".format
pd.set_option('display.max_rows', 500)

df = pd.DataFrame(table,columns =  list(map(make_short, ATRIBUTES_TABLE)))
display(df)

            

,track,domain,solution_value,sart_runtimes,cmp-tr,cmp-tr-ipcdiff,penalty-estimated,penalty-tr,penalty-eval,cmp-eval,cmp-eval-ipcdiff,estimated-error-coverage,estimated-error-runtime,coverage-estimated,coverage-tr
0,sat-1204,openstacks,69,False,5,-9,1.1,10,10,0,-21,12,3.6e+02,18,30
1,opt-1204,rovers,14,True,11,-2,1.5,2.3,3.1,18,-8,3,3.9,20,23
2,opt-1204,tpp,16,True,5,-6,1,2.3,1.8,22,-2,4,1.7,20,16
3,sat-1204,logistics,8.2,True,9,-3,2.5,2.5,1.9,13,-4,7,1.1,22,15
4,sat-1204,snake,4.4,True,9,-3,0.33,0,1.8,25,-2,0,1.4,16,16
5,opt-1204,elevators,46,True,14,-1,2.2,1.9,0.55,23,-3,2,1.5,20,18
6,opt-1204,openstacks,11,True,9,-3,1.3,2.4,1.3,21,-1,2,1.3,20,18
7,opt-1204,parking,3.8e+02,True,15,0,1.2,1,1.7,24,-4,2,1.6,22,20
8,opt-1204,scanalyzer,1.5e+02,True,5,-4,1.8,1.2,2.2,26,0,1,1.2,21,20
9,opt-1204,data-network,33,True,14,-1,1.9,1.7,1.9,26,-1,3,2.6,15,12


In [13]:
import pandas as pd

from IPython.display import display

def make_short(x): 
    return x.replace("comparisons", "cmp").replace("training", "tr")
ATRIBUTES_TABLE = ["track", "domain","runtimes-estimated", "runtimes-training", "runtimes-eval"]

table = []
for data in sorted(properties_dataset.values(), key = lambda x : x["comparisons-eval-ipcdiff"] + x["comparisons-training-ipcdiff"]): 
    table.append([data[atr] if atr in data else "-" for atr in ATRIBUTES_TABLE ])

pd.options.display.float_format = "{:,.2g}".format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100000)

df = pd.DataFrame(table,columns =  list(map(make_short, ATRIBUTES_TABLE)))
display(df)


,track,domain,runtimes-estimated,runtimes-tr,runtimes-eval
0,sat-1204,openstacks,"[0.99, 2.3, 6.7, 13.0, 62.0, 79.0, 210.0, 580.0, 1600.0, 4300.0, 12000.0, 31000.0, 85000.0, 34.0, 98.0, 310.0, 910.0, 2600.0, 7700.0, 22000.0, 59.0, 100.0, 250.0, 600.0, 1500.0, 3600.0, 9000.0, 22000.0, 54000.0, 130000.0]","[0.22, 0.23, 0.24, 0.24, 0.27, 0.25, 0.28, 0.27, 0.28, 0.33, 0.31, 0.35, 0.42, 0.24, 0.27, 0.29, 0.26, 0.3, 0.27, 0.33, 0.26, 0.25, 0.27, 0.28, 0.32, 0.35, 0.34, 0.33, 0.39, 0.37]","[0.42, 0.41, 0.43, 0.43, 0.44, 0.44, 0.46, 0.45, 0.49, 0.48, 0.48, 0.5, 0.55, 0.41, 0.44, 0.43, 0.44, 0.45, 0.5, 0.46, 0.45, 0.44, 0.44, 0.46, 0.47, 0.47, 0.49, 0.49, 0.52, 0.56]"
1,opt-1204,rovers,"[1.1, 1.4, 1.6, 1.6, 2.3, 2.5, 3.6, 3.8, 5.0, 5.7, 8.5, 9.3, 17.0, 46.0, 240.0, 780.0, 2500.0, 8200.0, 27000.0, 250.0, 460.0, 850.0, 1600.0, 2900.0, 5300.0, 9800.0, 18000.0, 33000.0, 61000.0, 110000.0]","[0.49, 1.34, 0.9, 3.15, 1.53, 1.65, 4.4, 8.41, 1.64, 3.15, 6.08, 23.37, 7.35, 13.2, 17.5, 31.06, 873.62, 2.58, unsolved, 12.7, unsolved, unsolved, 358.15, 13.43, 9.13, 983.57, unsolved, unsolved, unsolved, unsolved]","[0.52, 2.12, 0.66, 3.0, 2.38, 2.39, 2.22, 8.23, 2.5, 3.64, 6.85, 13.58, 10.52, 9.55, 17.38, 16.93, unsolved, 3.2, unsolved, 11.54, unsolved, unsolved, 137.97, 6.99, 12.57, unsolved, unsolved, unsolved, unsolved, unsolved]"
2,opt-1204,tpp,"[18.0, 29.0, 62.0, 180.0, 530.0, 1500.0, 4500.0, 13000.0, 39000.0, 1.1, 1.4, 1.9, 2.8, 3.7, 7.3, 11.0, 29.0, 37.0, 47.0, 150.0, 280.0, 550.0, 1100.0, 2100.0, 4100.0, 8100.0, 16000.0, 31000.0, 60000.0, 120000.0]","[17.55, 43.88, 36.97, 86.93, 1770.16, unsolved, unsolved, unsolved, unsolved, 0.5, 5.05, 2.12, 2.63, 2.76, 6.11, 14.43, 16.26, 21.38, 18.24, 26.27, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved]","[4.33, 4.91, 6.43, 11.58, 21.19, 17.12, 22.07, 33.75, 23.44, 0.53, 0.69, 1.26, 1.16, 1.34, 2.2, 3.14, 3.42, 4.85, 5.11, 6.49, 27.0, 15.16, 19.21, 32.53, 66.23, 35.99, 51.81, 42.89, 56.94, 61.87]"
3,sat-1204,logistics,"[4.8, 6.9, 9.4, 12.0, 14.0, 20.0, 180.0, 1700.0, 15000.0, 140000.0, 180.0, 1300.0, 9000.0, 64000.0, 180.0, 1500.0, 13000.0, 110000.0, 2.2, 3.0, 3.7, 5.0, 7.0, 9.2, 12.0, 14.0, 18.0, 180.0, 1800.0, 18000.0]","[4.86, 7.04, 9.96, 17.05, 24.56, 19.65, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, 2.28, 2.77, 3.81, 5.08, 7.15, 8.91, 12.11, 14.13, 18.29, unsolved, unsolved, unsolved]","[17.49, 17.83, 20.74, 24.26, 28.28, 35.82, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, 7.27, 10.58, 18.13, 14.33, 17.26, 20.0, 23.65, 26.14, 32.6, unsolved, unsolved, unsolved]"
4,sat-1204,snake,"[1.7, 3.4, 7.1, 11.0, 15.0, 26.0, 55.0, 140.0, 280.0, 530.0, 1000.0, 1900.0, 3600.0, 6800.0, 13000.0, 24000.0, 46000.0, 86000.0, 160000.0, 190.0, 310.0, 520.0, 850.0, 1400.0, 2300.0, 3800.0, 6300.0, 10000.0, 17000.0, 28000.0]","[1.82, 3.46, 5.8, 14.81, 9.1, 27.63, 59.2, 53.36, 225.74, 275.37, unsolved, unsolved, 371.47, 711.31, 742.16, unsolved, unsolved, unsolved, unsolved, 104.23, 476.91, unsolved, unsolved, unsolved, 271.74, unsolved, unsolved, unsolved, unsolved, unsolved]","[2.13, 2.98, 4.3, 10.81, 9.2, 20.91, 56.25, 70.53, 96.99, 113.47, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, unsolved, 60.22, 90.85, 80.51, 85.73, 192.08, 92.64, 144.03, unsolved, unsolved, unsolved, unsolved]"
5,opt-1204,elevators,"[1.1, 1.3, 1.8, 4.1, 12.0, 39.0, 170.0, 320.0, 990.0, 3100.0, 9500.0, 29000.0, 91000.0, 5.0, 36.0, 160.0, 320.0, 1400.0, 6600.0, 30000.0, 130000.0, 1.7, 5.6, 12.0, 60.0, 320.0, 1300.0, 5600.0, 23000.0, 97000.0]","[0.59, 1.81, 2.39, 2.3, 2.84, 10.05, 364.68, 57.63, 236.06, unsolved, unsolved, unsolved, unsolved, 7.92, 32.36, 239.23, 541.19, unsolved, unsolved, unsolved, unsolved, 1.79, 5.41, 53.76, 8.22, 347.18, unsolved, unsolved, unsolved, unsolved]","[0.65, 1.2, 3.12, 37.29, 2.49, 

In [20]:
properties_dataset["opt-1204", "tpp"]["coverage"]

Counter({'fd1906-bjolp': 2,
         'fd1906-blind': 2,
         'fd1906-fdss1-mas1-60s': 2,
         'fd1906-fdss1-mas2-60s': 2,
         'fd1906-lmcut': 2,
         'ipc2014-opt-symba1': 16,
         'ipc2018-opt-complementary2-3584mb': 15,
         'ipc2018-opt-decstar': 30,
         'ipc2018-opt-delfi-blind': 5,
         'ipc2018-opt-delfi-celmcut': 5,
         'ipc2018-opt-delfi-ipdb': 5,
         'ipc2018-opt-delfi-mas-miasm': 15,
         'ipc2018-opt-delfi-mas-sccdfp': 15,
         'ipc2018-opt-scorpion': 4})

In [21]:
properties_dataset["opt-1204", "tpp"]["coverage-ipc"]

Counter({'fd1906-bjolp': 6,
         'fd1906-blind': 6,
         'fd1906-fdss1-mas1-60s': 6,
         'fd1906-fdss1-mas2-60s': 7,
         'fd1906-lmcut': 7,
         'ipc2014-opt-symba1': 8,
         'ipc2018-opt-complementary2-3584mb': 14,
         'ipc2018-opt-decstar': 20,
         'ipc2018-opt-delfi-blind': 7,
         'ipc2018-opt-delfi-celmcut': 8,
         'ipc2018-opt-delfi-ipdb': 7,
         'ipc2018-opt-delfi-mas-miasm': 8,
         'ipc2018-opt-delfi-mas-sccdfp': 9,
         'ipc2018-opt-scorpion': 8})